In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"


client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
#list experiments
client.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/4', creation_time=1716604232795, experiment_id='4', last_update_time=1716604232795, lifecycle_stage='active', name='my-client-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/3', creation_time=1716426763664, experiment_id='3', last_update_time=1716426763664, lifecycle_stage='active', name='nyc-taxi-experiment3', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/2', creation_time=1716420347202, experiment_id='2', last_update_time=1716420347202, lifecycle_stage='active', name='nyc-taxi-experiment2', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/1', creation_time=1716350551855, experiment_id='1', last_update_time=1716350551855, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/

In [3]:
client.create_experiment(name="my-client-experiment")

MlflowException: Experiment(name=my-client-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage, creation_time, last_update_time) VALUES (?, ?, ?, ?, ?)]
[parameters: ('my-client-experiment', None, 'active', 1716611909875, 1716611909875)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='2',
    filter_string="metrics.rmse <6.4",
    run_view_type= ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

for run in runs:
    print(f"runs id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

runs id: 0b49e69c42574822ab43548f0bfc474d, rmse: 6.3267
runs id: 80fee7803c3a4f459faa94213dac78d0, rmse: 6.3499
runs id: 2ca5a0188f7b4825aaa9f2ab70f74ea3, rmse: 6.3683
runs id: 2bef0af08bd64a17a90b300a29731b54, rmse: 6.3849


In [6]:
#promote models
import mlflow 

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "0b49e69c42574822ab43548f0bfc474d"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1716611926397, current_stage='None', description=None, last_updated_timestamp=1716611926397, name='nyc-taxi-regressor', run_id='0b49e69c42574822ab43548f0bfc474d', run_link=None, source='/workspaces/mlops/02-experiment-tracking/mlruns/2/0b49e69c42574822ab43548f0bfc474d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [7]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1716601030603, description='', last_updated_timestamp=1716611926397, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1716611926397, current_stage='None', description=None, last_updated_timestamp=1716611926397, name='nyc-taxi-regressor', run_id='0b49e69c42574822ab43548f0bfc474d', run_link=None, source='/workspaces/mlops/02-experiment-tracking/mlruns/2/0b49e69c42574822ab43548f0bfc474d/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>], name='nyc-taxi-regressor', tags={}>]

In [8]:
model_name = 'nyc-taxi-regressor'

latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"{version.current_stage}")

None


/tmp/ipykernel_1044/1951477194.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.13.0/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [9]:
client.set_registered_model_alias(name=model_name, alias='development', version=3)